In [149]:
import pyspark_csv as pycsv
sc.addPyFile('pyspark_csv.py')
plaintext_rdd = sc.textFile('datos/caba_para_mapa.csv')
dataframe = pycsv.csvToDataFrame(sqlCtx, plaintext_rdd)
data = dataframe.rdd

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:97% !important; }</style>"))

# USE CABA PORQUE LSH TIRA ERROR CUANDO ENCUENTRA NANS

In [150]:
dataframe.dtypes

[('created_on', 'timestamp'),
 ('property_type', 'string'),
 ('place_name', 'string'),
 ('state_name', 'string'),
 ('lat-lon', 'string'),
 ('lat', 'double'),
 ('lon', 'double'),
 ('price', 'int'),
 ('currency', 'string'),
 ('price_aprox_local_currency', 'double'),
 ('price_aprox_usd', 'int'),
 ('surface_total_in_m2', 'int'),
 ('surface_covered_in_m2', 'int'),
 ('price_usd_per_m2', 'int'),
 ('price_per_m2', 'double'),
 ('floor', 'int'),
 ('rooms', 'int'),
 ('expenses', 'int'),
 ('properati_url', 'string'),
 ('description', 'string'),
 ('title', 'string'),
 ('dist_a_subte', 'double'),
 ('dist_a_tren', 'double'),
 ('dist_a_univ', 'double'),
 ('dist_a_villa', 'double'),
 ('dist_a_zona_anegada', 'double')]

In [153]:
from pyspark.ml.feature import VectorAssembler
#preparo las columnas para pasarle lsh
assembler = VectorAssembler(inputCols=["surface_total_in_m2","dist_a_subte"], outputCol="vector2")
dataframe = assembler.transform(dataframe)

In [154]:
dataframe.select("vector2").show()

+-----------+
|    vector2|
+-----------+
| [29.0,2.7]|
| [47.0,7.3]|
| [47.0,7.3]|
| [86.0,5.8]|
| [65.0,9.3]|
| [35.0,1.9]|
|[153.0,4.5]|
| [40.0,3.7]|
| [40.0,3.7]|
| [40.0,3.7]|
|[470.0,8.7]|
| [70.0,7.0]|
| [47.0,4.3]|
| [48.0,4.3]|
| [33.0,4.3]|
| [28.0,4.3]|
| [26.0,4.3]|
| [48.0,4.3]|
| [50.0,4.3]|
|[270.0,9.4]|
+-----------+
only showing top 20 rows



In [158]:
from pyspark.ml.feature import BucketedRandomProjectionLSH
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import col

key = Vectors.dense([51.3,0.62])

brp = BucketedRandomProjectionLSH(inputCol="vector2", outputCol="bucket", bucketLength=500.0,
                                  numHashTables=2)
model = brp.fit(dataframe)

# Compute the locality sensitive hashes for the input rows, then perform approximate nearest
# neighbor search.
# We could avoid computing hashes by passing in the already-transformed dataset, e.g.
# `model.approxNearestNeighbors(transformedA, key, 2)`
print("Approximately searching dfA for 20 nearest neighbors of",key)
resultado=model.approxNearestNeighbors(dataframe, key, 20,'distancia').select('place_name','vector2','bucket','distancia','price').show()


('Approximately searching dfA for 20 nearest neighbors of', DenseVector([51.3, 0.62]))
+------------+----------+--------------+-------------------+-------+
|  place_name|   vector2|        bucket|          distancia|  price|
+------------+----------+--------------+-------------------+-------+
|Barrio Norte|[51.0,0.7]|[[0.0], [0.0]]| 0.3104834939251977| 170000|
|   Balvanera|[51.0,0.7]|[[0.0], [0.0]]| 0.3104834939251977| 130000|
|    Recoleta|[51.0,0.7]|[[0.0], [0.0]]| 0.3104834939251977| 140000|
|    Recoleta|[51.0,0.5]|[[0.0], [0.0]]| 0.3231098884280676| 204585|
|Barrio Norte|[51.0,0.5]|[[0.0], [0.0]]| 0.3231098884280676| 152000|
|    Recoleta|[51.0,0.5]|[[0.0], [0.0]]| 0.3231098884280676| 178187|
|    Recoleta|[51.0,0.5]|[[0.0], [0.0]]| 0.3231098884280676| 181486|
|    Recoleta|[51.0,0.5]|[[0.0], [0.0]]| 0.3231098884280676| 184786|
|    Recoleta|[51.0,0.5]|[[0.0], [0.0]]| 0.3231098884280676| 188746|
|    Recoleta|[51.0,0.5]|[[0.0], [0.0]]| 0.3231098884280676| 192705|
|    Recoleta|[5

In [ ]:
resultado